In [1]:
import numpy as np 
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
from nltk.corpus import stopwords
#from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string
from sklearn.preprocessing import LabelEncoder


In [2]:
df = pd.read_csv('Taxml_Integers_Verified.csv', index_col=None, header=0,dtype={'item_name': str, 'description': str,'establishment_type': str, 'CAT_Name':str},usecols=['item_name','description','establishment_type','CAT_Name'])
#df_duplicates = df_integers[df_integers.duplicated(subset=['item_name','description','establishment_type'],keep=False)]


In [3]:
def cat_name_sorted(text):
    text=sorted(text.split(','))
    text=','.join([x for x in text])
    return text
def cat_name_without_temp(text):
    if ',TEMP_HEATED' in text:
        text=re.sub(',TEMP_HEATED','', text)
    if ',TEMP_UNHEATED' in text:
        text=re.sub(',TEMP_UNHEATED','', text)
    if ',TEMP_COLD' in text:
        text=re.sub(',TEMP_COLD','', text)
    if 'TEMP_HEATED' in text:
        text=re.sub('TEMP_HEATED','', text)
    if 'TEMP_UNHEATED' in text:
        text=re.sub('TEMP_UNHEATED','', text)
    if 'TEMP_COLD' in text:
        text=re.sub('TEMP_COLD','', text)
    else:
        pass
    #text=re.sub('TEMP_UNHEATED','', text)
    #text=re.sub('TEMP_UNHEATED','', text)
    return text
le=WordNetLemmatizer()
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    #print(tag,word)
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    #print( tag_dict.get(tag, wordnet.NOUN))

    return tag_dict.get(tag, wordnet.NOUN)

In [4]:
#df1 = df.sample(n=10000,random_state=1)
df1=df.copy()
df1['CAT_Name']=df1['CAT_Name'].progress_apply(lambda x: cat_name_sorted(x))
df1['CAT_Name']=df1['CAT_Name'].progress_apply(lambda x: cat_name_without_temp(x))
df1=df1.replace(r'^\s*$', np.nan, regex=True)
df1 = df1.dropna(subset=['CAT_Name'])
df1=df1.drop_duplicates(subset=['item_name','description','establishment_type','CAT_Name'],ignore_index=True,keep='first')
df1=df1.drop_duplicates(subset=['item_name','description','establishment_type'],ignore_index=True,keep='first')
df1['input_str'] = df1[['item_name', 'description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)

  0%|          | 0/175570 [00:00<?, ?it/s]

  0%|          | 0/175570 [00:00<?, ?it/s]

In [67]:
#pd.value_counts(df1['CAT_Name']).plot.bar()
#plt.title('cat histogram')
#plt.xlabel('Class')
#plt.ylabel('Frequency')
#df1['CAT_Name'].value_counts()

In [68]:
#from imblearn.over_sampling import SMOTE
#from collections import Counter

In [5]:
df1 = df1.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = list(set(stopwords.words('english')))
STOPWORDS.append('n')
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)# remove stopwors from text
    text =  " ".join([le.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text) if w not in string.punctuation])
    return text
df1['input_str'] = df1['input_str'].apply(clean_text)
df1['input_str'] = df1['input_str'].str.replace('\d+', '')
df1 = df1[df1['input_str'].apply(lambda x: len(str(x))>1)]


<ipython-input-5-8fbbb139b23f>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['input_str'] = df1['input_str'].str.replace('\d+', '')


In [6]:
df1.shape

(167440, 5)

In [ ]:
df1.to_csv('final_df1.csv')

In [71]:
MAX_SEQUENCE_LENGTH=-1
for i,rev in enumerate(df1['input_str']):
    tokens=rev.split()
    if(len(tokens)>MAX_SEQUENCE_LENGTH):
        MAX_SEQUENCE_LENGTH=len(tokens)
print(MAX_SEQUENCE_LENGTH)

293


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS=20000
# Max number of words in each complaint.
#MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1['input_str'].values)
word_index = tokenizer.word_index
word_counts = tokenizer.word_counts
#print('Found %s unique tokens.' % len(word_index))
#print('Found %s  tokens.' % len(word_counts))
#print(' the number of times words occur in the text corpus' % tokenizer.word_counts)
#print("The word docs",tokenizer.word_docs)

NameError: name 'nlp' is not defined

In [83]:
X = tokenizer.texts_to_sequences(df1['input_str'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (9540, 293)


In [84]:
X

array([[   0,    0,    0, ...,  306,  382,    1],
       [   0,    0,    0, ...,  914,   81,    1],
       [   0,    0,    0, ..., 6569, 1242,    1],
       ...,
       [   0,    0,    0, ...,   14,    4,    1],
       [   0,    0,    0, ...,   31, 2926,    1],
       [   0,    0,    0, ...,  659, 1544,    1]], dtype=int32)

In [85]:
Y = pd.get_dummies(df1['CAT_Name']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (9540, 161)


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (8586, 293)
Number transactions y_train dataset:  (8586, 161)
Number transactions X_test dataset:  (954, 293)
Number transactions y_test dataset:  (954, 161)


In [87]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(SpatialDropout1D(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 293, 100)          2000000   
                                                                 
 spatial_dropout1d_8 (Spatia  (None, 293, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 100)               80400     
                                                                 
 dense_8 (Dense)             (None, 161)               16261     
                                                                 
Total params: 2,096,661
Trainable params: 2,096,661
Non-trainable params: 0
_________________________________________________________________
None


In [88]:
epochs = 5
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.10,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
121/121 [==============================] - 40s 312ms/step - loss: 3.6268 - accuracy: 0.2060 - val_loss: 3.3326 - val_accuracy: 0.2247
Epoch 2/5
121/121 [==============================] - 36s 299ms/step - loss: 3.0800 - accuracy: 0.2741 - val_loss: 2.8305 - val_accuracy: 0.3108
Epoch 3/5
121/121 [==============================] - 37s 303ms/step - loss: 2.6540 - accuracy: 0.3809 - val_loss: 2.4779 - val_accuracy: 0.4319
Epoch 4/5
121/121 [==============================] - 36s 298ms/step - loss: 2.2962 - accuracy: 0.4803 - val_loss: 2.2345 - val_accuracy: 0.4843
Epoch 5/5
121/121 [==============================] - 36s 301ms/step - loss: 2.0148 - accuracy: 0.5373 - val_loss: 2.0521 - val_accuracy: 0.5204


In [89]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

30/30 [==============================] - 1s 41ms/step - loss: 1.9395 - accuracy: 0.5828
Test set
  Loss: 1.939
  Accuracy: 0.583


In [235]:
new_complaint = ['ej brandy grand blue vsop ml grocery']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
#labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
labels=list(set(df1['CAT_Name']))
print(new_complaint,labels[np.argmax(pred)])

['ej brandy grand blue vsop ml grocery'] CAT_PREPACKAGED_FOOD_HONEY


In [215]:
result=pd.DataFrame()
#seq = tokenizer.texts_to_sequences(X_test)
padded = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels=list(set(df1['CAT_Name']))
result['input']=tokenizer.sequences_to_texts(X_test)
result['original_cat']=labels[np.argmax(y_test)]
result['pred']=labels[np.argmax(pred)]


IndexError: list index out of range

In [217]:
pred

array([[0.04657657, 0.00423859, 0.00316973, ..., 0.00173439, 0.00862304,
        0.11367816],
       [0.04646255, 0.00416791, 0.00315172, ..., 0.00171035, 0.00858124,
        0.11470422],
       [0.04643726, 0.0041702 , 0.00314348, ..., 0.00170727, 0.00856694,
        0.11452761],
       ...,
       [0.04646696, 0.00418247, 0.00314667, ..., 0.00171043, 0.00859642,
        0.11440453],
       [0.04640952, 0.00418446, 0.00314905, ..., 0.00171597, 0.00857454,
        0.11437099],
       [0.04635875, 0.00418249, 0.00315512, ..., 0.00170454, 0.00861294,
        0.11461134]], dtype=float32)

In [201]:
labels[np.argmax(y_test[7])]

'CAT_TPP_PERFUMES'

In [210]:
y_test[74]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)